In [68]:
# import

import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import time
from datetime import datetime
from scipy.fft import fft
from scipy.interpolate import interp1d
import re

In [69]:
# define functions will be used

def get_time(s):
    hr = []; minute = []; sec = [];
    A = [m.start() for m in re.finditer('_', s)];
    hr = s[A[14] + 1] + s[A[14] + 2];
    minute = s[A[15] + 1] + s[A[15] + 2];
    sec = s[A[16] + 1]+s[A[16] + 2];
    return hr, minute, sec

def array_merge(A):
    tmp = np.array([]);
    A = np.array(A);
    for i in range(len(A)):
        tmp = np.append(tmp,A[i]);
    return tmp

def sort_by_index(X,ind):
    T = np.array(X);
    X = T[ind];
    return X

def check_data_exist(fn_type,nameX,nameY,nameZ,data):
    if(fn_type == 1):
        if not data:
            print('File error: Xfile is empty.');
            print('File error: Yfile is empty.');
            print('File error: Zfile is empty.');
            return 1
        return 0
    if(fn_type == 2):
        if not data:
            print(nameX+' has no time index value.')
            print(nameY+' has no time index value.')
            print(nameZ+' has no time index value.')
            return 1
        return 0
    

In [70]:
# Some basic settings

Sample_Rate = 400;
Sample_Period = 300;
r_normal = 0.001;
Amp_max = 0.01; # 600/60000
r_normal = 0.001;

foldername = '主機2020.03';
folderpwd = 'D:' + os.sep + '震動資料' + os.sep + foldername;
files = [f for f in os.listdir(folderpwd) if os.path.isfile(os.path.join(folderpwd, f))];

start_date = '2020_3_1';
end_date = '2020_3_31';
t1 = time.mktime(datetime.strptime(start_date,"%Y_%m_%d").timetuple());
t2 = time.mktime(datetime.strptime(end_date,"%Y_%m_%d").timetuple());
dd = (t2-t1)/86400 + 1;

f0 = np.linspace(1/Sample_Period, Sample_Rate/2, int(Sample_Period*Sample_Rate/2));

In [45]:
# Get X Y Z file and run fft

# run setting days
for dt in range(28,29): #range(dd)
    
    # Some settings
    Xfile = []; Yfile = []; Zfile = [];
    FX2 = []; FY2 = []; FZ2 = [];
    XA = []; YA = []; ZA = [];
    NA = []; N_original = []; t_old = [];
    #FXtmp = []; FYtmp = []; FZtmp = [];    
    #Q3X = []; Q3Y = []; Q3Z = [];
    #Q1X = []; Q1Y = []; Q1Z = [];
    #FA = {}; 
    
    Day_Stamp = datetime.fromtimestamp(t1 + dt * 86400);
    Day = Day_Stamp.strftime("%Y_%#m_%#d");
    Daycheck = Day + '_';
    
    
    # get file list in one day
    #for i in range(len(files)):
    #    if ((Daycheck in files[i]) & ('Ch_X' in files[i])):
    #        Xfile.append(files[i]);
    #    elif((Daycheck in files[i]) & ('Ch_Y' in files[i])):
    #        Yfile.append(files[i]);
    #    elif((Daycheck in files[i]) & ('Ch_Z' in files[i])):
    #        Zfile.append(files[i]);
    
    Xfile = [files[i] for i in range(len(files)) if ((Daycheck in files[i]) & ('Ch_X' in files[i]))]
    Yfile = [files[i] for i in range(len(files)) if ((Daycheck in files[i]) & ('Ch_Y' in files[i]))]
    Zfile = [files[i] for i in range(len(files)) if ((Daycheck in files[i]) & ('Ch_Z' in files[i]))]
    
    
    
    # check file existence
    #file_check = 0;
    #if (len(Xfile) == 0 ):
    #    print('File error: Xfile is empty.');
    #    break;
    #elif(len(Yfile) == 0):
    #    print('File error: Yfile is empty.');
    #    break;
    #elif(len(Zfile) == 0):
    #    print('File error: Zfile is empty.');
    #    break;
    if check_data_exist(1,_,_,_,min(len(Xfile),len(Yfile),len(Zfile))):
        break;

    time1 = time.time();
    # store data and run for each file
    for fileid in range(len(Xfile)): #range(len(Xfile))
        
        X = []; Y = []; Z = [];
        t_x = []; t_y = []; t_z = [];
        
        # get time info
        h, m, s = get_time(Xfile[fileid]);
        if('下' in Xfile[fileid]):
            c = 12;
        else:
            c = 0;
        h = str( int(h)%12 + c );
        current_date = Day + ':' + h + ':' + m + ':' + s;
        current_stamp = time.mktime(datetime.strptime(current_date,"%Y_%m_%d:%H:%M:%S").timetuple());
        
        #suppose three axis data have same time index and no single axis data loss
        # get three axis' data
        filepwd = folderpwd + os.sep + Xfile[fileid];
        loadx = np.loadtxt(filepwd, skiprows = 22, encoding = 'utf-8', delimiter = ';');
        t_x = [loadx[j][0] for j in range(len(loadx)) if(loadx[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        X = [loadx[j][1] for j in range(len(loadx)) if(loadx[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        
        #for j in range(len(loadx)):
        #    if(loadx[j][0] < ( (Sample_Rate * Sample_Period) + 10) ):
        #       t_x.append(loadx[j][0]);
        #        X.append(loadx[j][1]);

        filepwd = folderpwd + os.sep + Yfile[fileid];
        loady = np.loadtxt(filepwd, skiprows = 22, encoding = 'utf-8', delimiter = ';');
        t_y = t_x;#[loady[j][0] for j in range(len(loady)) if(loady[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        Y = [loady[j][1] for j in range(len(loady)) if(loady[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        
        #for j in range(len(loady)):
        #    if(loady[j][0] < ( (Sample_Rate * Sample_Period) + 10) ):
        #        t_y.append(loady[j][0]);
        #        Y.append(loady[j][1]);
            
        filepwd = folderpwd + os.sep + Zfile[fileid];
        loadz = np.loadtxt(filepwd, skiprows = 22, encoding = 'utf-8', delimiter = ';');
        t_z = t_x;#[loadz[j][0] for j in range(len(loadz)) if(loadz[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        Z = [loadz[j][1] for j in range(len(loadz)) if(loadz[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        
        #for j in range(len(loadz)):
        #    if(loadz[j][0] < ( (Sample_Rate * Sample_Period) + 10) ):
        #        t_z.append(loadz[j][0]);
        #        Z.append(loadz[j][1]);
        
        empty_controller = 0
        #if not t_x:
        #    print(Xfile[fileid], 'has no time index value.')
        #    empty_controller = 1
        #if not t_y:
        #    print(Yfile[fileid], 'has no time index value.')
        #    empty_controller = 1
        #if not t_z:
        #    print(Zfile[fileid], 'has no time index value.')
        #    empty_controller = 1
        if check_data_exist(2,Xfile[fileid],Yfile[fileid],Zfile[fileid],t_x):
            continue
        
        N_original.append(len(t_x));
        # interpolate        
        t = np.linspace(min(t_x), max(t_x), int(max(t_x) - min(t_x) + 1), dtype = int);
        X = interp1d(t_x,X)(t);
        Y = interp1d(t_y,Y)(t);
        Z = interp1d(t_z,Z)(t);
        
        t_old.append(current_stamp);
        XA.append(X);
        YA.append(Y);
        ZA.append(Z);
        
        #Q3X.append(np.quantile(X,1-r_normal));        
        #Q3Y.append(np.quantile(Y,1-r_normal));
        #Q3Z.append(np.quantile(Z,1-r_normal));
        #Q1X.append(np.quantile(X,r_normal));
        #Q1Y.append(np.quantile(Y,r_normal));
        #Q1Z.append(np.quantile(Z,r_normal));        
        
        N = len(t);
        NA.append(N);
        TT = N/Sample_Rate;
        f = np.linspace(1/TT,Sample_Rate/2,int(N/2));
        L = len(f);
        
        # three axis fft
        F = fft(X)/len(X);
        F = np.delete(F, list(range(L+1,len(F))))
        F = np.delete(F,0);
        FX = interp1d(f,abs(F))(f0[f0 > 1/TT]);
        FX = np.insert(FX,0,0);
        FX2.append(FX);
        
        F = fft(Y)/len(Y);
        F = np.delete(F, list(range(L+1,len(F))))
        F = np.delete(F,0);
        FY = interp1d(f,abs(F))(f0[f0 > 1/TT]);
        FY = np.insert(FY,0,0);
        FY2.append(FY);
        
        F = fft(Z)/len(Z);
        F = np.delete(F, list(range(L+1,len(F))))
        F = np.delete(F,0);
        FZ = interp1d(f,abs(F))(f0[f0 > 1/TT]);
        FZ = np.insert(FZ,0,0);
        FZ2.append(FZ);
        
    time2 = time.time();
    tt = sorted(t_old);
    ttindex = np.argsort(t_old);
    
    # for i in [FX2,FY2,FZ2,Q3X,Q3Y,Q3Z,Q1X,Q1Y,Q1Z,NA,XA,YA,ZA]:
    #     T = np.array(i);i = T[ttindex];
    T = np.array(FX2);FX2 = T[ttindex];
    T = np.array(FY2);FY2 = T[ttindex];
    T = np.array(FZ2);FZ2 = T[ttindex];
    T = np.array(NA);NA = T[ttindex];
    T = np.array(XA);XA = T[ttindex];
    T = np.array(YA);YA = T[ttindex];
    T = np.array(ZA);ZA = T[ttindex];
    # T = np.array(Q3X);Q3X = T[ttindex];
    # T = np.array(Q3Y);Q3Y = T[ttindex];
    # T = np.array(Q3Z);Q3Z = T[ttindex];
    # T = np.array(Q1X);Q1X = T[ttindex];
    # T = np.array(Q1Y);Q1Y = T[ttindex];
    # T = np.array(Q1Z);Q1Z = T[ttindex];    
    
    # check time index existence
    if (len(tt) == 0):
        print('Error: no time index.');
        break;
    
    tlen = len(tt);
    time_label_list = ['00:00', '03:00', '06:00', '09:00', '12:00', 
                       '15:00', '18:00','21:00','00:00'];
    Ttick = np.linspace(0,tlen,9);
    
    time3 = time.time();
    
    # plot time-freq for each axis
    figX, axX = plt.subplots();
    img = axX.imshow(np.matrix.transpose(FX2) , cmap = 'jet', vmin = 0, vmax = Amp_max,
                    aspect = 'auto', origin = 'lower', extent = [0,tlen,0,200]);
    axX.set_xticks(Ttick);
    axX.set_xticklabels(time_label_list);
    figX.colorbar(img);
    figname = Day + '_X';
    axX.title.set_text(figname);
    plt.savefig(figname,dpi=600);
    #plt.imsave();
    
    figY, axY = plt.subplots();
    img = axY.imshow(np.matrix.transpose(FY2) , cmap = 'jet', vmin = 0, vmax = Amp_max,
                    aspect = 'auto', origin = 'lower', extent = [0,tlen,0,200]);
    axY.set_xticks(Ttick);
    axY.set_xticklabels(time_label_list);
    figY.colorbar(img);
    figname = Day + '_Y';
    axY.title.set_text(figname);
    plt.savefig(figname,dpi=600);
    
    figZ, axZ = plt.subplots();
    img = axZ.imshow(np.matrix.transpose(FZ2) , cmap = 'jet', vmin = 0, vmax = Amp_max,
                    aspect = 'auto', origin = 'lower', extent = [0,tlen,0,200]);
    axZ.set_xticks(Ttick);
    axZ.set_xticklabels(time_label_list);
    figZ.colorbar(img);
    figname = Day + '_Z';
    axZ.title.set_text(figname);
    plt.savefig(figname,dpi=600);
    
    # plot raw data
    XA = array_merge(XA);
    YA = array_merge(YA);
    ZA = array_merge(ZA);
    xlen = len(XA);
    ylen = len(YA);
    zlen = len(ZA);
    Xtick = np.linspace(0,xlen,9);
    Ytick = np.linspace(0,ylen,9);
    Ztick = np.linspace(0,zlen,9);
    
    figT = plt.figure();
    axT = figT.add_subplot(111);
    axT.spines['top'].set_color('none');
    axT.spines['bottom'].set_color('none');
    axT.spines['left'].set_color('none');
    axT.spines['right'].set_color('none');
    axT.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False);
    axT.title.set_text(Day+'_T');
    
    axT1 = figT.add_subplot(311);
    plt.plot(XA,'b');
    plt.xlabel('time');
    plt.ylabel('X');
    plt.xlim([0,xlen]);
    plt.ylim([-3,3]);
    axT1.set_xticks(Xtick);
    axT1.set_xticklabels(time_label_list);
    
    axT2 = figT.add_subplot(312);
    plt.plot(YA,'g');
    axT2.set_xticks(Ytick);
    axT2.set_xticklabels(time_label_list);
    plt.xlabel('time');
    plt.ylabel('Y');
    plt.xlim([0,ylen]);
    plt.ylim([-3,3]);
    
    axT3 = figT.add_subplot(313);
    plt.plot(ZA,'r');
    axT3.set_xticks(Ztick);
    axT3.set_xticklabels(time_label_list);
    plt.xlabel('time');
    plt.ylabel('Z');
    plt.xlim([0,zlen]);
    plt.ylim([-3,3]);
    plt.tight_layout();
    figname = Day + '_T';
    plt.savefig(figname,dpi=600);
    
    figN = plt.figure();
    axN = figN.add_subplot();
    plt.plot(np.array(NA)/120000);
    axN.set_xticks(np.linspace(0,tlen,9));
    axN.set_xticklabels(time_label_list);
    plt.xlim([0,len(tt)]);
    plt.ylim([0,1.2]);
    plt.xlabel('time');
    plt.ylabel('Effective Data Rate');
    figname = Day + '_N';
    axN.title.set_text(figname);
    plt.savefig(figname,dpi=600);
    time4 = time.time();

In [ ]:
F = fft(X)/len(X);
F = np.delete(F, list(range(L+1,len(F))))
F = np.delete(F,0);
FX = interp1d(f,abs(F))(f0[f0 > 1/TT]);
FX = np.insert(FX,0,0);
FX2.append(FX);

In [67]:
len(X)

120000

for loop：746.7089166641235

list comprehension：

    for i in ['FX2','FY2','FZ2','Q3X','Q3Y','Q3Z','Q1X','Q1Y','Q1Z']:
        T = np.array(i);
        i = T[ttindex];
    

    l = len(tt);
    figX, ax = plt.subplots();
    img = ax.imshow(np.matrix.transpose(FX2) , cmap = 'jet', vmin = 0, vmax = 0.01,
                    aspect = 'auto', origin = 'lower', extent = [0,l,0,200]);
    x_label_list = ['00:00', '03:00', '06:00', '09:00', '12:00', 
                    '15:00', '18:00','21:00','00:00'];
    Xtick = [0,l/8,l/4,3*l/8,l/2,5*l/8,3*l/4,7*l/8,l]
    ax.set_xticks(Xtick);
    ax.set_xticklabels(x_label_list);
    figX.colorbar(img);